# HW2: Exploratory Data Analysis




<span style="color:green">    - Antônio Luís Sombra de Medeiros</span>  
<span style="color:green"> - João Marcos Amorim </span>  
<span style="color:green">     - Renato Aranha</span> 




Guidelines:

- What question do you want to answer?

    
- The classification of Cartola FC follows the table of classification of Brasileirão?

   <span style="color:blue">  
A: We rebuilt the final classification of the championship following its criteria of scoring the same and built the classification after the last of the Brasileirão by the total points made by the club in Cartola FC.  </span>
       
       
- Where have you obtained the data?

   <span style="color:blue"> A: The scraped data from the official Cartola [website] can be found [here]:</span>
    
    
- How were your data manipulation and cleaning?

 <span style="color:blue"> A: We emphasize these aspects on the notebook body </span>
    
    
- Visualize your data from multiple perspectives and indicate if you found outliers, patterns, trends.

  <span style="color:blue"> A: Graphs were created to help understand the variation in player's score and price per position and per year, helping to understand if there was a significant change in the years and we checked the variation of scores and price per position. </span>


- Indicate if your visualizations help you to answer the questions or if you have a more clear idea what path you should follow to corroborate your hypothesis. 


- Give a conclusion of your analysis. 


[website]: http://globoesporte.globo.com/cartola-fc/
[here]: https://github.com/thevtm/CartolaFCDados

# 0. DATA

* Our data can be found at: [https://github.com/thevtm/CartolaFCDados](https://github.com/thevtm/CartolaFCDados)  
It's the record from 2014 to 2017 of Cartola Data (the fantasy game)  


* We have information from players,teams, games and "scouts", which are points for what happened in each game. For example:  
 * We have data for how many goals a players did in a game, how many goals the goalkeeper of each team avoided...


* We basically work with 6 csvs (for each year from 2014 to 2017)


    1. Atletas.csv   (Data of each player)
    2. Clubes.csv    (Data of each Team)
    3. Partidas.csv  (Data of each game)
    4. Posicoes.csv  (Table with all positions possible for a player)
    5. Scouts.csv    (Game data and observations, points etc..)
    6. Status.csv    (Table with all the possible status for a player - Suspenso, Contundido, etc. - )
    
    
    
    
##### METADATA



For Atletas csv:

| Field   | Description |
| ------  | -----------|
| ID       | ID of player.|
| Apelido  | How player is called (nickname) |
| ClubeID  | ID of players' team |
| PosicaoID| ID of Players' position in field (1 = goalkepper,2 = defense,..., 4 = midfielder, 5= attacker, 6 = coach) |
| Ano      | Year the player was playing for this team and in this position |


For Clubes csv:

| Field    | Description|
| ------   | -----------|
| ID       | Team's ID  |
| Nome     | Team's name|
|Abreviacao| Team's Acronym  |
| Slug     | Team's slug |


For Partidas.csv:

| Field        | Description                        |
| ------       | -----------                        |
| Ano          | Game's Year                        |
| CasaID       | Home Team's ID                     |
| ID           | Game's ID                          |
| PlacarCasa   | Number of goals of home team      |
| PlacarVisitante| Number of goals of visiting team |
| Resultado     | Winner of game                    |
| Rodada        | Championship's Round number (for that game) |
| VisitanteID   | Visitor Team's ID                 |


For Posicoes.csv:

| Field    | Description|
| ------   | -----------|
| ID       | Position's ID  |
| Nome     | Position's name|
|Abreviacao| Position's Acronym  |


For Scouts.csv: 

| Field       | Description|
| ------      | -----------|
|Rodada       | Round      |
|ClubeID      | Team's ID  |
|AtletaID     | Players' ID|
|Participou   | If player played on that game or not|
|Pontos       | Number of Cartola points for that game  |
|PontosMedia  | Média de pontos do atleta até esta rodada (inclue rodada atual)  |
|Preco        | Preço do atleta nesta rodada  |
|PrecoVariacao|  Variação do preço da rodada passada para esta |
|FS           | Faltas sofridas  |
|PE           |Passes errados  |
|A            | Assistências |
|FT           | Finalizações na trave  |
|FD           | Finalizações defendidas  |
|FF           | Finalizações para fora  |
|G            | Gols  |
|I            | Impedimentos  |
|PP           | Penaltis perdidos  |
|RB           | Roubadas de bola  |
|FC           | Faltas cometidas  |
|GC           | Gols contras  |
|CA           | Cartões Amarelos  |
|CV           | Cartões Vermelhos  |
|SG           | Jogo sem sofrer gols |
|DD           | Defesas dificeis  |
|DP           | Defesa de penaltis |
|GS           | Gols sofridos  |
|Ano          | Year  |
- **Rodada**: Rodada em que o scout ocorreu
- **ClubeID**: ID do clube do atleta
- **AtletaID**: ID do atleta
- **Participou**: `TRUE` se o atleta participou do jogo, `FALSE` se ficou no banco
- **Pontos**: Pontuação do atleta nesta rodada
- **PontosMedia**: Média de pontos do atleta até está rodada (inclue rodada atual)
- **Preco**: Preço do atleta nesta rodada
- **PrecoVariacao**: Variação do preço da rodada passada para está
- **FS**: Faltas sofridas
- **PE**: Passes errados
- **A**: Assistências
- **FT**: Finalizações na trave
- **FD**: Finalizações defendidas
- **FF**: Finalizações para fora
- **G**: Gols
- **I**: Impedimentos
- **PP**: Penaltis perdidos
- **RB**: Roubadas de bola
- **FC**: Faltas cometidas
- **GC**: Gols contras
- **CA**: Cartões Amarelos
- **CV**: Cartões Vermelhos
- **SG**: Jogo sem sofrer gols
- **DD**: Defesas dificeis
- **DP**: Defesa de penaltis
- **GS**: Gols sofridos



For Status.csv:


| Field    | Description|
| ------   | -----------|
| ID       | Status's ID  |
| Nome     | Status's name|



# 1. LOADING DATA

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
sns.set(color_codes=True)


import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
cwd = os.getcwd()
cwd

'C:\\Users\\Renato Aranha\\Documents\\Mestrado\\Foundation_of_DS\\foundations_of_data_science'

### MERGING DATA FOR ALL YEARS (2014-2017)

In [3]:
atletas_all_years = pd.DataFrame()
clubes_all_years = pd.DataFrame()
partidas_all_years = pd.DataFrame()
posicoes_all_years = pd.DataFrame()
scouts_all_years = pd.DataFrame()
status_all_years = pd.DataFrame()


for i in range(2014, 2018):
    
    path_atl = os.path.join(cwd, str(i)+"/Atletas.csv")    
    path_club = os.path.join(cwd, str(i)+"/Clubes.csv")    
    path_part = os.path.join(cwd, str(i)+"/Partidas.csv")    
    path_pos = os.path.join(cwd, str(i)+"/Posicoes.csv")   
    path_sct = os.path.join(cwd, str(i)+"/Scouts.csv")    
    path_stts = os.path.join(cwd, str(i)+"/Status.csv")
    
    atletas = pd.read_csv(path_atl)
    atletas['Ano'] = i
    clubes = pd.read_csv(path_club)
    #clubes['Ano'] = i
    partidas = pd.read_csv(path_part)
    partidas['Ano'] = i
    posicoes = pd.read_csv(path_pos)
    #posicoes['Ano'] = i
    scouts = pd.read_csv(path_sct)
    scouts['Ano'] = i
    status = pd.read_csv(path_stts)
    #status['Ano'] = i
    
    atletas_all_years = atletas_all_years.append(atletas)
    clubes_all_years = clubes_all_years.append(clubes).drop_duplicates()
    partidas_all_years = partidas_all_years.append(partidas)
    posicoes_all_years = posicoes_all_years.append(posicoes).drop_duplicates()
    scouts_all_years = scouts_all_years.append(scouts)
    status_all_years = status_all_years.append(status).drop_duplicates()

### View merged tables with all years

In [4]:
partidas_all_years.head()

,Ano,CasaID,ID,PlacarCasa,PlacarVisitante,Resultado,Rodada,VisitanteID
0,2014,262,179872,0.0,0.0,Empate,1,290
1,2014,266,179873,3.0,0.0,Casa,1,316
2,2014,276,179874,3.0,0.0,Casa,1,263
3,2014,277,179875,1.0,1.0,Empate,1,292
4,2014,293,179876,1.0,0.0,Casa,1,284


In [5]:
clubes_all_years.head()

,ID,Nome,Abreviacao,Slug
0,262,Flamengo,FLA,flamengo
1,263,Botafogo,BOT,botafogo
2,264,Corinthians,COR,corinthians
3,265,Bahia,BAH,bahia
4,266,Fluminense,FLU,fluminense


# 2. DATA CLEANING AND MANIPULATION

In this stage we create functions to generate consolidated dataframes with games and teams. 
We noticed that we have duplicated observations for Sport Club (SPO, which is also SPT) and so we removed these data from our dataframe.
Also we generate a classification table, adding the points for each victory (3 points), tie (1 point) and loss(0 points) to each teams' scores.

In [6]:
def gera_partidas_com_clubes(partidas_all_years, clubes_all_years):
    partidas_all_years = partidas_all_years.merge(clubes_all_years[['ID','Abreviacao']], how='inner', left_on='CasaID',\
                         right_on='ID')
    partidas_all_years = partidas_all_years.merge(clubes_all_years[['ID','Abreviacao']], how='inner', left_on='VisitanteID',\
                         right_on='ID')
    partidas_all_years = partidas_all_years.drop(labels=['ID_y','ID'], axis = 1)
    partidas_com_clubes = partidas_all_years.rename(columns={'ID_x':'ID_partida','Abreviacao_x':'abrev_clube_casa','Abreviacao_y':'abrev_clube_fora'})
    return partidas_com_clubes

def remove_erros_SPO(partidas_com_clubes):
    a = partidas_com_clubes.copy().drop_duplicates()
    temp = pd.DataFrame(a.groupby(['ID_partida'])['ID_partida'].count())
    temp2 = temp[temp.ID_partida == 2]
    
    a = a[(a.ID_partida.isin(temp2.index))]
    a = a[(a.abrev_clube_casa == 'SPO') | (a.abrev_clube_fora == 'SPO')]
    
    final = partidas_com_clubes[~partidas_com_clubes.index.isin(a.index)]
    partidas_com_clubes_corrigido = final.drop_duplicates()
    
    return partidas_com_clubes_corrigido

def adiciona_pontuacao(partidas_com_clubes_corrigido):
    def pts_casa(casa, fora):
        if casa > fora:
            a = 3
        elif casa < fora:
            a = 0
        else:
            a = 1
        return a

    def pts_fora(casa, fora):
        if casa > fora:
            a = 0
        elif casa < fora:
            a = 3
        else:
            a = 1
        return a
    
    a = partidas_com_clubes_corrigido.copy().drop_duplicates().reset_index()
    a['pts_casa'] = a.apply(lambda x: pts_casa(x.PlacarCasa, x.PlacarVisitante), axis=1)
    a['pts_fora'] = a.apply(lambda x: pts_fora(x.PlacarCasa, x.PlacarVisitante), axis=1)
    
    partidas_com_pontuacao = a
    
    return partidas_com_pontuacao

def gera_classificacao(partidas_com_pontuacao):
    b = partidas_com_pontuacao.copy()
    casa = pd.DataFrame(b.groupby(['Ano','abrev_clube_casa'])['pts_casa'].sum())
    fora = pd.DataFrame(b.groupby(['Ano','abrev_clube_fora'])['pts_fora'].sum())
    tabela = pd.concat([casa, fora], axis=1)
    tabela['pts_total'] = tabela.pts_casa + tabela.pts_fora
    tabela = tabela.reset_index().sort_values(by=['Ano','pts_total'], ascending = [True, False])
    return tabela

In [7]:
partidas_com_clubes = gera_partidas_com_clubes(partidas_all_years, clubes_all_years).drop_duplicates()

partidas_com_clubes_corrigido = remove_erros_SPO(partidas_com_clubes)

partidas_com_pontuacao = adiciona_pontuacao(partidas_com_clubes_corrigido)

tabela = gera_classificacao(partidas_com_pontuacao)

tabela

,Ano,abrev_clube_casa,pts_casa,pts_fora,pts_total
8,2014,CRU,47,33,80
17,2014,SAO,39,31,70
6,2014,COR,42,27,69
14,2014,INT,45,24,69
2,2014,CAM,41,21,62
11,2014,FLU,39,22,61
13,2014,GRE,40,21,61
3,2014,CAP,35,19,54
16,2014,SAN,33,20,53
10,2014,FLA,35,17,52


This tables represents the final scores/classification for each years' championship

In [8]:
#Games(Matches) tables without duplicated values

partidas_com_clubes_corrigido

,Ano,CasaID,ID_partida,PlacarCasa,PlacarVisitante,Resultado,Rodada,VisitanteID,abrev_clube_casa,abrev_clube_fora
0,2014,262,179872,0.0,0.0,Empate,1,290,FLA,GOI
2,2015,262,190387,4.0,1.0,Casa,34,290,FLA,GOI
4,2014,266,180066,2.0,0.0,Casa,13,290,FLU,GOI
6,2015,266,190326,2.0,0.0,Casa,28,290,FLU,GOI
8,2014,276,180145,3.0,0.0,Casa,31,290,SAO,GOI
10,2015,276,190233,0.0,3.0,Visitante,19,290,SAO,GOI
12,2014,277,179998,2.0,0.0,Casa,25,290,SAN,GOI
14,2015,277,190355,3.0,1.0,Casa,31,290,SAN,GOI
16,2014,293,180179,1.0,0.0,Casa,37,290,CAP,GOI
18,2015,293,190261,3.0,0.0,Casa,21,290,CAP,GOI


In [9]:
partidas_com_clubes_corrigido.to_csv('partidas_all_years.csv', sep='\t')

In [10]:
atletas_all_years = atletas_all_years.drop_duplicates(['ID','Ano']) 

In [11]:
scouts_all_years.shape

(125016, 27)

In [12]:
clubes_all_years[['ID','Nome']].drop_duplicates().sort_values(by='ID')

,ID,Nome
0,1,Outros
0,262,Flamengo
1,263,Botafogo
2,264,Corinthians
3,265,Bahia
4,266,Fluminense
3,267,Vasco
5,275,Palmeiras
6,276,São Paulo
7,277,Santos


### Adding player name, team and position

In [13]:
scouts_all_years = scouts_all_years.merge(atletas_all_years[['ID','Apelido','PosicaoID','Ano']], left_on=['AtletaID','Ano'],\
                         right_on=['ID','Ano'],how = 'inner')
print(scouts_all_years.shape)
scouts_all_years = scouts_all_years.drop_duplicates()
print(scouts_all_years.shape)
scouts_all_years = scouts_all_years.merge(posicoes_all_years[['ID','Abreviacao']], left_on = 'PosicaoID',right_on = 'ID')
print(scouts_all_years.shape)
scouts_all_years = scouts_all_years.drop(labels=['ID_y','ID_x'], axis = 1)
print(scouts_all_years.shape)
scouts_all_years = scouts_all_years.merge(clubes_all_years[['ID','Nome']].drop_duplicates().sort_values(by='ID').iloc[1:,:]
                                          ,left_on='ClubeID',right_on='ID')
print(scouts_all_years.shape)



(125016, 30)
(125016, 30)
(125016, 32)
(125016, 30)
(125009, 32)


In [14]:
scouts_all_years.columns

Index(['Rodada', 'ClubeID', 'AtletaID', 'Participou', 'Pontos', 'PontosMedia',
       'Preco', 'PrecoVariacao', 'FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I',
       'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS', 'Ano',
       'Apelido', 'PosicaoID', 'Abreviacao', 'ID', 'Nome'],
      dtype='object')

### Droping players that din't player the match

In [15]:
print('Tamanho antes de deixar apenas quem jogou')
print(scouts_all_years.shape)
jogaram_all_years = scouts_all_years[scouts_all_years.Participou==True]
print('Tamanho depois de deixar apenas quem jogou')
print(jogaram_all_years.shape)

Tamanho antes de deixar apenas quem jogou
(125009, 32)
Tamanho depois de deixar apenas quem jogou
(43926, 32)


In [16]:
# libraries
import pandas as pd
import matplotlib.pyplot as plt

# Create a dataframe
value1=np.random.uniform(size=20)
value2=value1+np.random.uniform(size=20)/4
#df = pd.DataFrame({'group':list(map(chr, range(65, 85))), 'value1':value1 , 'value2':value2 })
df = pd.DataFrame({'group':list(map(chr, range(65, 85))), 'rank':export['rank'] , 'rank_cartola':export['rank_cartola'] })

# Reorder it following the values of the first value:
ordered_df = df.sort_values(by='rank', ascending = False)
my_range=range(1,len(df.index)+1)
 
# The vertical plot is made using the hline function
# I load the saeaborn library only to benefit the nice looking feature
import seaborn as sns
fig_size[0], fig_size[1] = 10, 9

labels = tabela_unificada.Abreviacao[tabela_unificada['Ano'] == 2014]

'''for label, x, y in zip(labels, export['rank'], export['rank_cartola']):
    plt.annotate(
        label,
        xy=(x, y), 
        xytext=(-20, 20),
        textcoords='offset points', ha='left', va='top',
        bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.2),
        arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))'''
    
plt.vlines(x=my_range, ymin=ordered_df['rank'], ymax=ordered_df['rank_cartola'], color='grey', alpha=0.8)
plt.scatter( my_range, ordered_df['rank'], color='blue', alpha=1, label='Real rank')
plt.scatter( my_range, ordered_df['rank_cartola'], color='red', alpha=0.4 , label='Cartola rank')
plt.rcParams["figure.figsize"] = fig_size
plt.legend(loc=2, fontsize = 15)
 
# Add title and axis names
plt.yticks(my_range, ordered_df['rank'])
#plt.xticks(my_range, ordered_df['rank'])
plt.title("Comparison of real rank and cartola rank - 2014", loc='center', fontsize = 15)
plt.ylabel('Classification', fontsize = 15)
#plt.xlabel('Group')

NameError: name 'export' is not defined

# Linear Regression price and points by positions

Can we find some linear relationship between players performance and their prices?

The conclusion would be the same if we analize this relationship by player position?

The plots below show the scores of players in each round by their prices, and even by eyesight, we can judge that there is no clear pattern.

PS.1: The players prices changes in every round, based on their previous round performance.  
PS.2: We are using data from the championships of all years (2014 to 2016) and there is no inflation.

In [ ]:
#fig, ax = plt.subplots()
#fig.set_size_inches(16,10)
plt.figure(figsize=(24,4))
vis = sns.lmplot(x="Preco", y="Pontos", data=jogaram_all_years.sample(10000),col='Abreviacao',legend=True,fit_reg=True,\
                 col_wrap=2, size=4,aspect=0.9)
#vis.axes.set_title('Scatter plot of points by price',fontsize=20)
#vis.tick_params(labelsize=15)
vis.set(xlabel='Price (Cartoletas)', ylabel='Points')
plt.show()

### Are there any differences in the score distribution of players, by position?

we can see that the medians of the positions GOL, LAT and ZAG are higher that other positions. 
As can be observed, the coaches have the lowest dispersion in the data, but they are far from reaching a score higher than 10 points. Goalkeepers are the players with the most scattered score, without a punctuation tendency

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10,6)
position = ['TEC', 'GOL', 'LAT', 'ZAG', 'MEI', 'ATA']
vis = sns.violinplot(data = jogaram_all_years, x='Abreviacao', y="Pontos", palette="GnBu_d",\
                    order=position)
vis.set(yticks=np.arange(-20,40,10))
vis.axes.set_title('Violin plot of Players points by positions', fontsize=20)
vis.tick_params(labelsize=15)
plt.show()

In [ ]:
jogaram_all_years.loc[:,['Abreviacao','Pontos']].groupby('Abreviacao').describe()

#### Are there any differences in the price distribution of players, by position?


We can see that similar to the score the coach price is what less varies compared to other positions, and the position of goalkeeper in addition to more dispersed is the most expensive of the game

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16,10)
position = ['TEC', 'GOL', 'LAT', 'ZAG', 'MEI', 'ATA']
vis = sns.violinplot(data = jogaram_all_years, x='Abreviacao', y="Preco", palette="GnBu_d",\
                    order=position)
vis.set(yticks=np.arange(0,40,10))
vis.axes.set_title('Violin plot of players price by positions', fontsize=20)
vis.tick_params(labelsize=15)
plt.show()

In [ ]:
jogaram_all_years.loc[:,['Abreviacao','Preco']].groupby('Abreviacao').describe()

### Are there any differences in the variation of price distribution of players, by position?

As we can see the coaches are the ones who have the lowest price variation from one round to another compared to the other positions.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16,10)
vis = sns.violinplot(data = jogaram_all_years, x='Abreviacao', y="PrecoVariacao", palette="GnBu_d")
vis.set(yticks=np.arange(-20,30,10))
vis.axes.set_title('Violin plot of player price variation by positions',fontsize=20)
vis.tick_params(labelsize=15)
plt.show()



In [ ]:
jogaram_all_years.loc[:,['Abreviacao','PrecoVariacao']].groupby('Abreviacao').describe()

In [ ]:
jogaram_all_years.head()

### filtering the 4 best and 4 worst teams of each year

In [ ]:
l2014 = ['Cruzeiro','São Paulo','Internacional','Corinthians','Vitória','Bahia','Botafogo','Criciúma']
l2015 = ['Corinthians','Atlético-MG','Grêmio','São Paulo','Avaí','Vasco','Goiás','Joinville']
l2016 = ['Palmeiras','Santos','Flamengo','Atlético-MG','Internacional','Figueirense','Santa Cruz','América-MG']
l2017 = ['Corinthians','Palmeiras','Santos','Grêmio','Coritiba','Avaí','Ponte Preta','Atlético-GO']

In [ ]:
aux = jogaram_all_years[(jogaram_all_years.Ano ==2014) & jogaram_all_years.Nome.isin(l2014) ]

### Is there any price difference between the top 4 and 4 worst teams?
we can see that the players of the team with the best performance in the league had a higher price in the game during the year.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16,10)
vis = sns.violinplot(data = aux, x='Nome', y="Preco", palette="GnBu_d",order=l2014)
vis.set(yticks=np.arange(0,40,10))
vis.axes.set_title('Violin plot of players price of the 4 best and 4 worst teams in 2014',fontsize=20)
vis.tick_params(labelsize=15)
vis.set_xlabel('Teams')
plt.show()

In [ ]:
aux[['Nome','Preco']].groupby('Nome').describe()

### Is there any difference in score between the top 4 and 4 worst teams?
We can see that the top 4 teams have the median higher than the 4 worst teams and the third quartile of the relegationed teams have been far from the third quartile of the top 4 teams.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16,10)
vis = sns.violinplot(data = aux, x='Nome', y="Pontos", palette="GnBu_d",order=l2014)
vis.set(yticks=np.arange(0,40,10))
vis.axes.set_title('Violin plot of players pontos of the 4 best and 4 worst teams in 2014',fontsize=20)
vis.tick_params(labelsize=15)
vis.set_xlabel('Teams')
plt.show()

In [ ]:
aux[['Nome','Pontos']].groupby('Nome').describe()

In [ ]:
aux = jogaram_all_years[(jogaram_all_years.Ano ==2015) & jogaram_all_years.Nome.isin(l2015) ]

### Is there any price difference between the top 4 and 4 worst teams?
we can see that the players of the team with the best performance in the league had a higher price in the game during the year.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16,10)
vis = sns.violinplot(data = aux, x='Nome', y="Preco", palette="GnBu_d",order=l2015)
vis.set(yticks=np.arange(0,40,10))
vis.axes.set_title('Violin plot of players price of the 4 best and 4 worst teams in 2015',fontsize=20)
vis.tick_params(labelsize=15)
vis.set_xlabel('Teams')
plt.show()

In [ ]:
aux[['Nome','Preco']].groupby('Nome').describe()

# 2015

### Is there any difference in score between the top 4 and 4 worst teams?
We can see that the top 4 teams have the median higher than the 4 worst teams and the third quartile of the relegationed teams have been far from the third quartile of the top 4 teams.with Goiás performing similarly to the team.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16,10)
vis = sns.violinplot(data = aux, x='Nome', y="Pontos", palette="GnBu_d",order=l2015)
vis.set(yticks=np.arange(0,40,10))
vis.axes.set_title('Violin plot of players pontos of the 4 best and 4 worst teams in 2015',fontsize=20)
vis.tick_params(labelsize=15)
vis.set_xlabel('Teams')
plt.show()

In [ ]:
aux[['Nome','Pontos']].groupby('Nome').describe()

# 2016

In [ ]:
aux = jogaram_all_years[(jogaram_all_years.Ano ==2016) & jogaram_all_years.Nome.isin(l2016) ]

### Is there any price difference between the top 4 and 4 worst teams?
we can see that the players of the team with the best performance in the league had a higher price in the game during the year.
With the Santa Cruz having a player like outlier, that was the Graffiti.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16,10)
vis = sns.violinplot(data = aux, x='Nome', y="Preco", palette="GnBu_d",order=l2016)
vis.set(yticks=np.arange(0,40,10))
vis.axes.set_title('Violin plot of players price of the 4 best and 4 worst teams in 2016',fontsize=20)
vis.tick_params(labelsize=15)
vis.set_xlabel('Teams')
plt.show()

In [ ]:
aux[['Nome','Preco']].groupby('Nome').describe()

### Is there any difference in score between the top 4 and 4 worst teams?
We can see that the top 4 teams have the median higher than the 4 worst teams and the third quartile of the relegationed teams have been far from the third quartile of the top 4 teams.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16,10)
vis = sns.violinplot(data = aux, x='Nome', y="Pontos", palette="GnBu_d",order=l2016)
vis.set(yticks=np.arange(0,40,10))
vis.axes.set_title('Violin plot of players pontos of the 4 best and 4 worst teams in 2016',fontsize=20)
vis.tick_params(labelsize=15)
vis.set_xlabel('Teams')
plt.show()

In [ ]:
aux[['Nome','Pontos']].groupby('Nome').describe()

# 2017

In [ ]:
aux = jogaram_all_years[(jogaram_all_years.Ano ==2017) & jogaram_all_years.Nome.isin(l2017) ]

### Is there any price difference between the top 4 and 4 worst teams?
# ARANHA ANALISA ESTE AQUI

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16,10)
vis = sns.violinplot(data = aux, x='Nome', y="Preco", palette="GnBu_d",order=l2017)
vis.set(yticks=np.arange(0,40,10))
vis.axes.set_title('Violin plot of players price of the 4 best and 4 worst teams in 2017',fontsize=20)
vis.tick_params(labelsize=15)
vis.set_xlabel('Teams')
plt.show()

In [ ]:
aux[['Nome','Preco']].groupby('Nome').describe()

### Is there any difference in score between the top 4 and 4 worst teams?
We can see that the top 4 teams have the median higher than the 4 worst teams and the third quartile of the relegationed teams have been far from the third quartile of the top 4 teams.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16,10)
vis = sns.violinplot(data = aux, x='Nome', y="Pontos", palette="GnBu_d",order=l2017)
vis.set(yticks=np.arange(0,40,10))
vis.axes.set_title('Violin plot of players pontos of the 4 best and 4 worst teams in 2017',fontsize=20)
vis.tick_params(labelsize=15)
vis.set_xlabel('Teams')
plt.show()

In [ ]:
aux[['Nome','Pontos']].groupby('Nome').describe()

### Are there any differences in the price distribution of players, by years?

As we can see there is no difference in the distribution of players' price over the years.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16,10)
vis = sns.violinplot(data = jogaram_all_years, x='Ano', y="Preco", palette="GnBu_d")
vis.set(yticks=np.arange(0,40,10))
vis.axes.set_title('Violin plot of players price by years',fontsize=20)
vis.tick_params(labelsize=15)
plt.show()


In [ ]:
jogaram_all_years.loc[:,['Ano','Preco']].groupby('Ano').describe()

### Are there any differences in the points distribution of players, by years?

As we can see there is no difference in the distribution of players' scores over the years.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16,10)
vis = sns.violinplot(data = jogaram_all_years, x='Ano', y="Pontos", palette="GnBu_d")
vis.set(yticks=np.arange(-20,40,10))
vis.axes.set_title('Violin plot of players points by years',fontsize=20)
vis.tick_params(labelsize=15)
plt.show()

In [ ]:
jogaram_all_years.loc[:,['Ano','Pontos']].groupby('Ano').describe()

### Are there any differences in the variation of price distribution of players, by years?


We can see that there is no difference between the years

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16,10)
vis = sns.violinplot(data = jogaram_all_years, x='Ano', y="PrecoVariacao", palette="GnBu_d")
vis.set(yticks=np.arange(-20,30,10))
vis.axes.set_title('Violin plot of player variation price by years',fontsize=20)
vis.tick_params(labelsize=15)
plt.show()

In [ ]:
jogaram_all_years.groupby(['ClubeID','Ano']).mean().reset_index().head(10)

### GENERATING TABLES WITH ATHLETES

In [ ]:
atletas_all_years = atletas_all_years.drop_duplicates(['ID'])

In [ ]:
atletas_all_years.head()

In [ ]:
clubes_all_years.head()

In [ ]:
posicoes_all_years

In [ ]:
atletas_all_years = atletas_all_years.merge(clubes_all_years[['ID','Abreviacao','Nome']], how='inner', left_on='ClubeID',\
                         right_on='ID')

atletas_all_years = atletas_all_years.merge(posicoes_all_years[['ID','Abreviacao']], how='inner', left_on='PosicaoID',\
                         right_on='ID')

atletas_all_years = atletas_all_years.drop(labels=['ID_y','ID'], axis = 1)

In [ ]:
atletas_all_years = atletas_all_years.rename(columns={'ID_x':'ID_jogador','Nome':'nome_clube','Abreviacao_x':'abrev_clube',\
                                                     'Abreviacao_y':'posicao'})

In [ ]:
atletas_all_years = atletas_all_years.drop_duplicates(['ID_jogador'])

In [ ]:
atletas_all_years.head()

In [ ]:
atletas_all_years = atletas_all_years.drop_duplicates()

In [ ]:
atletas_all_years.to_csv('atletas_all_years.csv', sep='\t')

In [ ]:
scouts_all_years.head()

### Generating club scores round by round

In [ ]:
treinadores = atletas_all_years.ID_jogador[atletas_all_years.PosicaoID == 6]

cols1 = ['Ano','Rodada', 'ClubeID', 'AtletaID', 'Participou', 'Pontos']
tabela_cartola = scouts_all_years[scouts_all_years.AtletaID.isin(treinadores)][cols1]

cols2 = ['Ano','Rodada', 'ClubeID', 'AtletaID', 'Participou', 'Pontos', 'Abreviacao']
tabela_cartola = tabela_cartola.merge(clubes_all_years[['ID','Abreviacao','Nome']], how='inner', left_on='ClubeID',\
                         right_on='ID')[cols2]

cols3 = ['Ano','Rodada', 'ClubeID', 'AtletaID', 'Apelido','Participou', 'Pontos', 'Abreviacao']
tabela_cartola = tabela_cartola.merge(atletas_all_years[['ID_jogador','Apelido']], how='inner', left_on='AtletaID',\
                         right_on='ID_jogador')[cols3]

tabela_cartola = tabela_cartola.sort_values(by=['Ano', 'ClubeID', 'Rodada'])

tabela_cartola['Apelido'] = tabela_cartola.apply(lambda x: str(x.Apelido).lower(), axis=1)

tabela_cartola = tabela_cartola.drop_duplicates()

tabela_cartola

In [ ]:
tabela_quase_final = pd.DataFrame(tabela_cartola.groupby(['Ano','Abreviacao'])['Pontos'].mean())
tabela_final = tabela_quase_final.reset_index().sort_values(by=['Ano','Pontos'], ascending = [True, False])
tabela_final

In [ ]:
tabela

In [ ]:
cols_final = ['Ano', 'Abreviacao', 'pts_total', 'Pontos']
tabela_unificada = tabela.merge(tabela_final, how = 'inner', left_on = ['Ano','abrev_clube_casa'], right_on=['Ano','Abreviacao'])[cols_final]
tabela_unificada = tabela_unificada.rename(columns={'pts_total':'pts_real','Pontos':'pts_cartola'})
tabela_unificada.head()

In [ ]:
tabela_unificada['rank'] = np.where(tabela_unificada['Ano']==2014, [i+1 for i in range(0,len(tabela_unificada))], '')
tabela_unificada.head()

In [ ]:
tabela_unificada.to_csv('tabela_unificada.csv', sep='\t')

In [ ]:
#tabela_unificada = tabela_unificada.sort_values(by=['Ano','pts_cartola'], ascending=[True, False])
#tabela_unificada['rank_cartola'] = np.where(tabela_unificada['Ano']==2014, [i+1 for i in range(0, len(tabela_unificada))], '')
#tabela_unificada = tabela_unificada.sort_values(by=['Ano','pts_real'], ascending=[True, False])
tabela_unificada.Abreviacao[tabela_unificada['Ano'] == 2014]

In [ ]:
rank_real = tabela_unificada[tabela_unificada.Ano == 2014]['rank']
rank_cartola = tabela_unificada[tabela_unificada.Ano == 2014]['rank_cartola']

In [ ]:
export = tabela_unificada[tabela_unificada.Ano == 2014][['Ano','rank','rank_cartola']]
export = export[['rank','rank_cartola','Ano']]
export = export.astype(np.float)
#export.to_csv('export.csv', sep='\t')

In [ ]:
export

In [ ]:
# libraries
import pandas as pd
import matplotlib.pyplot as plt

# Create a dataframe
value1=np.random.uniform(size=20)
value2=value1+np.random.uniform(size=20)/4
#df = pd.DataFrame({'group':list(map(chr, range(65, 85))), 'value1':value1 , 'value2':value2 })
df = pd.DataFrame({'group':list(map(chr, range(65, 85))), 'rank':export['rank'] , 'rank_cartola':export['rank_cartola'] })

# Reorder it following the values of the first value:
ordered_df = df.sort_values(by='rank', ascending = False)
my_range=range(1,len(df.index)+1)
 
# The vertical plot is made using the hline function
# I load the saeaborn library only to benefit the nice looking feature
import seaborn as sns
fig_size[0], fig_size[1] = 10, 9

labels = tabela_unificada.Abreviacao[tabela_unificada['Ano'] == 2014]

'''for label, x, y in zip(labels, export['rank'], export['rank_cartola']):
    plt.annotate(
        label,
        xy=(x, y), 
        xytext=(-20, 20),
        textcoords='offset points', ha='left', va='top',
        bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.2),
        arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))'''
    
plt.vlines(x=my_range, ymin=ordered_df['rank'], ymax=ordered_df['rank_cartola'], color='grey', alpha=0.8)
plt.scatter( my_range, ordered_df['rank'], color='blue', alpha=1, label='Real rank')
plt.scatter( my_range, ordered_df['rank_cartola'], color='red', alpha=0.4 , label='Cartola rank')
plt.rcParams["figure.figsize"] = fig_size
plt.legend(loc=2, fontsize = 15)
 
# Add title and axis names
plt.yticks(my_range, ordered_df['rank'])
#plt.xticks(my_range, ordered_df['rank'])
plt.title("Comparison of real rank and cartola rank - 2014", loc='center', fontsize = 15)
plt.ylabel('Classification', fontsize = 15)
#plt.xlabel('Group')

# Exploratory Data Analysis

### Plot of .....

In [ ]:
tabela_unificada[['pts_real','pts_cartola']][tabela_unificada.Ano==2014].plot()

In [ ]:
atletas_all_years.info()

In [ ]:
partidas_all_years.info()

### Joint Distribution ClubeID x PosicaoID


In [ ]:
atletas_all_years.head()

In [ ]:
sns.jointplot(x="ClubeID", y="PosicaoID", data=atletas_all_years[atletas_all_years.Ano == 2017], size=6, xlim=[250,400]);

Here we plot the joint plot of ClubeID (teams) and PositionID (players positions in field)


Notice that:

* We have more players on "MEIA" position 
* The team with more players is the one with ClubeID nearest 260.

### Histogram (number of observations for each value on each columns) of Athletes_all_years table

OBS: PosicaoID é a posição do jogador em campo (1 = zagueiro, 2 = lateral, ..., 5 = atacante, 6 = técnico)

In [ ]:
atletas_all_years.head()

In [ ]:
atletas_all_years.hist(figsize=(8, 8), bins=50, xlabelsize=8, ylabelsize=8);

In [ ]:
atletas_all_years.sample(10).sort_values("ID_jogador")

### Interactive Plots on TABLEAU

* Number of Players,with positions, by year: [link](https://public.tableau.com/profile/renato.aranha#!/vizhome/qtd_jog_por_posicao/Planilha1)
